# American Binomial Model

In [1]:
import numpy as np

class Option:
    def __init__(self, strike, expiry):
        self.strike = strike
        self.expiry = expiry

    def value(self):
        pass

class VanillaCallOption(Option):
    def value(self, spot):
        return np.maximum(0.0, spot - self.strike)

class VanillaPutOption(Option):
    def value(self, spot):
        return np.maximum(0.0, self.strike - spot)

def AmericanBinomialPricer(option, spot, rate, volatility, dividend, steps):
    numNodes = steps + 1
    dt = option.expiry / steps
    u = np.exp(((rate - dividend) * dt) + volatility * np.sqrt(dt)) 
    d = np.exp(((rate - dividend) * dt) - volatility * np.sqrt(dt))
    pu = (np.exp(rate*dt)-d)/(u-d)
    pd = 1 - pu
    
    discount = np.exp(-rate * dt)
    dpu = discount * pu
    dpd = discount * pd
    spotT = np.zeros(numNodes)
    putT = np.zeros(numNodes)
    
    for i in range(numNodes):
        spotT[i] = spot * (u ** (steps - i)) * (d ** i)
        putT[i] += option.value(spotT[i])

    for i in range((steps-1), -1, -1):
        for j in range(i+1):
            putT[j] = dpu * putT[j] + dpd * putT[j+1]
            spotT[j] = spotT[j]/u
            putT[j] = np.maximum(putT[j], option.value(spotT[j]))
            
    return putT[0]
        
spot = 41.0
strike = 40.0
rate = 0.08
expiry = 1.0
volatility = 0.30
dividend = 0.0
steps = 3

theCall = VanillaCallOption(strike, expiry)
CallPrice = AmericanBinomialPricer(theCall, spot, rate, volatility, dividend, steps)

thePut = VanillaPutOption(strike, expiry)
PutPrice = AmericanBinomialPricer(thePut, spot, rate, volatility, dividend, steps)

print("The price of the following call option is", CallPrice)
print("The price of the following put option is", PutPrice)

The price of the following call option is 7.07385326128
The price of the following put option is 3.29294758539
